# BookRAG

## Установка библиотек

In [117]:
# !pip install streamlit streamlit-chat

In [118]:
# !pip install langchain

In [119]:
# !pip install -U langchain-community

In [120]:
# !pip install sentence-transformers

In [121]:
# !pip install faiss-gpu

In [122]:
# !pip install openai

In [123]:
# !pip install langchain_openai

In [124]:
# !pip install pypdf

In [125]:
# !pip install tiktoken

## Препроцессинг данных

In [106]:
from langchain_community.document_loaders import PyPDFLoader
import re
from typing import List, Dict

In [107]:
# Путь к PDF файлу
PDF_FILE_PATH = '/kaggle/input/karamazovy/dostoevskiy_bratya_karamazovy.pdf'

# Загрузка страниц из PDF
loader = PyPDFLoader(PDF_FILE_PATH)
pages = loader.load()[7:]

# # Обработка текста для каждой страницы
# for i in range(len(pages)):
#     # Текущий текст страницы
#     text = pages[i].page_content
    
#     # Убираем переносы слов (например, "припоминае-\nмого" -> "припоминаемого")
#     text = re.sub(r'-\n', '', text)
    
#     # Заменяем разрывы строк на пробелы (например, "\n" -> " ")
#     text = re.sub(r'\n', ' ', text)
    
#     # Удаляем лишние пробелы
#     text = re.sub(r'\s+', ' ', text).strip()
    
#     # Удаляем подстроку с названием книги и автором
#     text = re.sub(r'Ф\.?\s*М\.?\s*Достоевский\.?\s*«Братья Карамазовы»', '', text, flags=re.IGNORECASE)
    
#     # Перезаписываем текст страницы
#     pages[i].page_content = text

# Проверка количества страниц после обработки
total_pages = len(pages)
print(f"Всего страниц: {total_pages}")

Всего страниц: 503


In [108]:
pages[0]

Document(metadata={'source': '/kaggle/input/karamazovy/dostoevskiy_bratya_karamazovy.pdf', 'page': 7}, page_content='Ф.  М.  Достоевский.  «Братья Карамазовы»\n8\n \nЧасть первая\n \n \nКнига первая\nИстория одной семейки\n \n \nI\nФедор Павлович Карамазов\n \nАлексей Федорович Карамазов был третьим сыном помещика нашего уезда Федора\nПавловича Карамазова, столь известного в свое время (да и теперь еще у нас припоминае-\nмого) по трагической и темной кончине своей, приключившейся ровно тринадцать лет назад\nи о которой сообщу в своем месте. Теперь же скажу об этом «помещике» (как его у нас назы-\nвали, хотя он всю жизнь совсем почти не жил в своем поместье) лишь то, что это был стран-\nный тип, довольно часто, однако, встречающийся, именно тип человека не только дрянного\nи развратного, но вместе с тем и бестолкового, – но из таких, однако, бестолковых, кото-\nрые умеют отлично обделывать свои имущественные делишки, и только, кажется, одни эти.\nФедор Павлович, например, начал почти чт

In [109]:
def split_text_into_chunks_with_metadata(pages, chunk_size=500):
    current_book = None
    current_chapter = None
    current_chapter_title = None
    is_next_title = True

    chunks = []
    for page in pages:
        text = page.page_content
        page_number = page.metadata['page'] + 1

        # Поиск текущей книги, главы и названия
        for line in text.splitlines():
            book_match = re.match(r"^\s*Книга\s+(\w+)", line)
            chapter_match = re.match(r"^\s*([IVXLCDM]+)\s*$", line)
            
            if current_book and is_next_title:
                current_chapter_title = line.strip()
                is_next_title = False
                
            if book_match:
                current_book = f"Книга {book_match.group(1)}"
                
            if chapter_match:
                current_chapter = f"Глава {chapter_match.group(1)}"
                is_next_title = True 

        # Разбиваем текст на чанки
        for i in range(0, len(text), chunk_size):
            chunk = text[i:i + chunk_size]
    
            # Убираем переносы слов (например, "припоминае-\nмого" -> "припоминаемого")
            chunk_text = re.sub(r'-\n', '', chunk)
            
            # Заменяем разрывы строк на пробелы (например, "\n" -> " ")
            chunk_text = re.sub(r'\n', ' ', chunk_text)
            
            # Удаляем лишние пробелы
            chunk_text = re.sub(r'\s+', ' ', chunk_text).strip()
            
            # Удаляем подстроку с названием книги и автором
            chunk_text = re.sub(r'Ф. М. Достоевский. «Братья Карамазовы»', '', chunk_text, flags=re.IGNORECASE)
    
            chunks.append({
                'chunk': f"Текст чанка {chunk_text}. | Источник: номер страницы {page_number}, {current_book}, {current_chapter}",
                'page': page_number,
                'book': current_book,
                'chapter': current_chapter,
                # 'chapter_title': current_chapter_title
            })
    return chunks

chunks = split_text_into_chunks_with_metadata(pages)
print(f"Всего чанков: {len(chunks)}")

Всего чанков: 3925


## Реализация RAG

In [110]:
import faiss
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [111]:
api_key = input("Введите API ключ: ")

In [112]:
# Инициализация модели эмбеддингов
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", openai_api_key=api_key)

# Создание списка текстов для индексации
texts = [chunk['chunk'] for chunk in chunks]
metadatas = [chunk for chunk in chunks]

# Создание векторного хранилища FAISS
vector_store = FAISS.from_texts(texts, embeddings, metadatas=metadatas)

# Сохранение индекса (опционально)
vector_store.save_local("faiss_index")

In [113]:
llm = ChatOpenAI(
    model="gpt-4o",           # Specify the GPT-4o model
    temperature=0.00,         # Low creativity for precise answers
    openai_api_key=api_key,   # Your OpenAI API key
    max_tokens=2048           # Adjust as needed
)

In [114]:
from langchain.prompts import PromptTemplate

template = """Вы — ассистент, специализирующийся на предоставлении точных ответов на вопросы по книге "Братья Карамазовы" Фёдора Достоевского.

Используйте предоставленные отрывки из книги для формирования ответов. Обязательно:

1. Приводите полный и развернутый ответ на вопрос.
2. Затем приводите точные цитаты из текста книги, подтверждающие ваш ответ, извлекая их из контекста.
3. Указывайте источник информации.


Контекст:
{context}

Вопрос: {question}

Ответ:
1. [Развернутый и полный ответ на вопрос]
2. [Цитата из текста: "..."]
3. [Источник: страница page, book, chapter]
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

In [115]:
# Создание цепочки Вопрос-Ответ с использованием RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Использование простого объединения
    retriever=vector_store.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True,  # Добавляем возврат источников
    chain_type_kwargs={"prompt": prompt}
)

In [128]:
# def chatbot():
#     print("Чат-бот по книге 'Братья Карамазовы'. Задайте свой вопрос или введите 'выход' для завершения.")
#     while True:
#         try:
#             user_input = input("Вы: ")
#             if user_input.lower() in ['выход', 'exit', 'quit']:
#                 print("Чат-бот: До свидания!")
#                 break
#             response = qa_chain.invoke(user_input)
#             # print(response["source_documents"])
#             result = response['result']
#             print(result)
#         except Exception as e:
#             print(f"Чат-бот: Извините, произошла ошибка: {e}")

# if __name__ == "__main__":
#     chatbot()

## Streamlit

In [ ]:
import streamlit as st
from streamlit_chat import message

# Инициализация приложения Streamlit
st.set_page_config(page_title="Чат-бот: Братья Карамазовы", layout="centered")
st.title("Чат-бот: Братья Карамазовы")
st.write("Задайте любой вопрос о книге 'Братья Карамазовы'. Введите 'выход' для завершения чата.")

# Поле для ввода OpenAI API ключа
st.sidebar.header("Настройки API")
api_key = st.sidebar.text_input("Введите ваш OpenAI API ключ:", type="password", placeholder="Введите ваш API ключ здесь...")
st.sidebar.write("Пожалуйста, предоставьте ваш API ключ перед началом чата. Если ключ не предоставлен, бот вернёт стандартный ответ.")

# Состояние сессии для хранения истории сообщений
if 'messages' not in st.session_state:
    st.session_state.messages = []

# Отображение существующих сообщений
for i, chat in enumerate(st.session_state.messages):
    if chat['role'] == 'user':
        message(chat['content'], is_user=True, key=f"user_{i}")
    else:
        message(chat['content'], key=f"bot_{i}")

# Поле ввода для вопроса пользователя
user_input = st.text_input("Ваш вопрос:", placeholder="Введите ваш вопрос здесь и нажмите Enter...")

if user_input:
    if user_input.lower() in ['выход', 'exit', 'quit']:
        st.write("Чат завершён. Обновите страницу для начала нового разговора.")
    else:
        # Добавление сообщения пользователя в историю
        st.session_state.messages.append({"role": "user", "content": user_input})

        try:
            # Генерация ответа с использованием qa_chain
            response = qa_chain.invoke(user_input)
        except Exception as e:
            response = f"Извините, произошла ошибка: {e}"

        # Добавление ответа бота в историю
        st.session_state.messages.append({"role": "bot", "content": response})

        # Отображение новых сообщений
        message(user_input, is_user=True, key=f"user_{len(st.session_state.messages) - 2}")
        message(response, key=f"bot_{len(st.session_state.messages) - 1}")